In [3]:
import nltk
import numpy as py
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

import re
import numpy as np
import pandas as pd
from pprint import pprint

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from collections import Counter
from gensim.test.utils import datapath
import pickle
from jieba import analyse

In [8]:
def retrieve_feature_label(path):
    df = pd.read_pickle(path)
    return df['total_notes'], df['label']

def clear(x):
    tfidf = analyse.extract_tags
    stop_words = set(stopwords.words('english'))
    x=str(x)
    x=x.lower()
    word_tokens=nltk.word_tokenize(x)
    word_tokens=[lemmatizer.lemmatize(x) for x in word_tokens]
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    keywords = tfidf(x,topK = 10)
    return keywords 

def check_token(X):
	all_token=[]
	for x in X:
		all_token += x
		print(Counter(all_token).most_common(100))

def update_stop_words(addlist):
	stop_words = set(stopwords.words('english'))
	for x in addlist:
		stop_words.add(x)
	return stop_words

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts, stop_words):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts, trigram_mod, bigram_mod ):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(nlp, texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


def process_X(X):
	addlist=['.',',','cw','spoke','!','?']
	stop_words = update_stop_words(addlist)
	data = X.values.tolist()
	data_words = list(sent_to_words(data))
	# Build the bigram and trigram models
	bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
	trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
	# Faster way to get a sentence clubbed as a trigram/bigram
	bigram_mod = gensim.models.phrases.Phraser(bigram)
	trigram_mod = gensim.models.phrases.Phraser(trigram)
	print("Finish trigram")
	print("-------------------------------------------")
	# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
	nlp = spacy.load('en',disable=['parser', 'ner']) 	# python3 -m spacy download en
	data_words_nostops = remove_stopwords(data_words, stop_words) 	# Remove Stop Words
	#data_words_trigrams = make_trigrams(data_words_nostops, trigram_mod, bigram_mod)	 	# Form Bigrams
	data_lemmatized = lemmatization(nlp, data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'ADV']) 	# Do lemmatization keeping only noun, adj, vb, adv
	id2word = corpora.Dictionary(data_lemmatized)
	# Create Corpus
	texts = data_lemmatized
	# Term Document Frequency
	corpus = [id2word.doc2bow(text) for text in texts]
	print("Finish building corpus")
	print("-------------------------------------------")
	lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

	return corpus, lda_model


def save_model(corpus, model):
	model.save('lda3.model')
	with open("corpus3.txt", "wb") as p:   #Pickling
		pickle.dump(corpus, p)


In [9]:
X, Y = retrieve_feature_label("train.pickle")
X = X.apply(clear)
corpus, model = process_X(X)
save_model(corpus, model)

Finish trigram
-------------------------------------------
Finish building corpus
-------------------------------------------


In [11]:
lda_model = gensim.models.ldamodel.LdaModel.load('lda3.model')
print(lda_model.print_topics())
with open("corpus3.txt", "rb") as fp:   # Unpickling
    corpus = pickle.load(fp)
predicted_labels = lda_model.get_document_topics(corpus)
new_label = []
prob = []
for i in predicted_labels:
    label,probability = max(i, key=lambda x:x[1])
    new_label.append(label)
    prob.append(probability)
new_label = np.array(new_label)

[(0, '0.149*"visit" + 0.129*"mom" + 0.091*"call" + 0.081*"text" + 0.072*"tomorrow" + 0.033*"due" + 0.029*"house" + 0.018*"afternoon" + 0.017*"currently" + 0.015*"pm"'), (1, '0.222*"writer" + 0.175*"staffing" + 0.066*"worker" + 0.065*"unit" + 0.061*"still" + 0.036*"inform" + 0.029*"vacation" + 0.025*"voucher" + 0.024*"attend" + 0.019*"curfew"'), (2, '0.131*"meeting" + 0.122*"case" + 0.095*"also" + 0.065*"office" + 0.059*"week" + 0.049*"parent" + 0.047*"next" + 0.045*"plan" + 0.036*"job" + 0.026*"discharge"'), (3, '0.166*"client" + 0.061*"treatment" + 0.058*"report" + 0.056*"able" + 0.050*"date" + 0.038*"clinical" + 0.036*"initial" + 0.033*"information" + 0.028*"informed" + 0.023*"future"'), (4, '0.338*"mother" + 0.054*"history" + 0.039*"appointment" + 0.029*"placement" + 0.029*"level" + 0.025*"angel" + 0.022*"last" + 0.021*"night" + 0.021*"sister" + 0.020*"foster"'), (5, '0.117*"phone" + 0.093*"day" + 0.050*"community" + 0.044*"present" + 0.043*"new" + 0.039*"message" + 0.039*"morning" 

In [12]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
pyLDAvis.save_html(vis, 'lda3.html')